In [25]:
import pydicom
import SimpleITK as sitk

def load_dicom_series(directory):
    reader = sitk.ImageSeriesReader()
    dicom_names = reader.GetGDCMSeriesFileNames(directory)
    reader.SetFileNames(dicom_names)
    image = reader.Execute()
    return image



ct_image = load_dicom_series('G:/Dokumente/MedicalIM/Pahnato4Radiometrics/manifest-1681918831927/CT-Phantom4Radiomics/SPHNQA4IQI/01-10-2017-101-ThoraxCTD01THROUTINENEU Adult-98669/101.000000-Harmonized b 10 mGy 2.0  I30f  3 axial - 1-71842')


In [26]:
import os
Folders = "G:/Dokumente/MedicalIM/Pahnato4Radiometrics/manifest-1681918831927/CT-Phantom4Radiomics/SPHNQA4IQI/"

Folders
ct_image = load_dicom_series('G:/Dokumente/MedicalIM/Pahnato4Radiometrics/manifest-1681918831927/CT-Phantom4Radiomics/SPHNQA4IQI/01-10-2017-101-ThoraxCTD01THROUTINENEU Adult-98669/101.000000-Harmonized b 10 mGy 2.0  I30f  3 axial - 1-71842')


In [27]:
def load_dicom_file(file_path):
    return sitk.ReadImage(file_path)

mask = load_dicom_file('G:/Dokumente/MedicalIM/Pahnato4Radiometrics/manifest-1681918831927/CT-Phantom4Radiomics/SPHNQA4IQI/01-10-2017-101-ThoraxCTD01THROUTINENEU Adult-98669/101.000000-Harmonized b 10 mGy 2.0  I30f  3 axial - 1 - SEG-35360/1-1.dcm')

In [28]:
mask

<SimpleITK.SimpleITK.Image; proxy of <Swig Object of type 'std::vector< itk::simple::Image >::value_type *' at 0x000001CA47E979C0> >

In [29]:
from radiomics import featureextractor


def resample_image_to_reference(image, reference_image):
    resampler = sitk.ResampleImageFilter()
    resampler.SetReferenceImage(reference_image)
    resampler.SetInterpolator(sitk.sitkNearestNeighbor)
    resampler.SetTransform(sitk.Transform())
    resampler.SetOutputSpacing(reference_image.GetSpacing())
    resampler.SetSize(reference_image.GetSize())
    resampler.SetOutputDirection(reference_image.GetDirection())
    resampler.SetOutputOrigin(reference_image.GetOrigin())
    return resampler.Execute(image)

# Resample mask to match CT image
resampled_mask = resample_image_to_reference(mask, ct_image)


# Initialize feature extractor (assuming you have a params.yaml)
params = 'C:/Users/Gabriel/OneDrive/Dokumente/GitHub/Medical_image/git_extraction/params/qa4iqi_params.yml'
extractor = featureextractor.RadiomicsFeatureExtractor(params)

features = extractor.execute(ct_image, resampled_mask, label=255)

for feature_name, feature_value in features.items():
    print(f"{feature_name}: {feature_value}")

diagnostics_Versions_PyRadiomics: 0+unknown
diagnostics_Versions_Numpy: 1.26.2
diagnostics_Versions_SimpleITK: 2.1.1.2
diagnostics_Versions_PyWavelet: 1.2.0
diagnostics_Versions_Python: 3.10.13
diagnostics_Configuration_Settings: {'minimumROIDimensions': 2, 'minimumROISize': None, 'normalize': False, 'normalizeScale': 1, 'removeOutliers': None, 'resampledPixelSpacing': None, 'interpolator': 'sitkBSpline', 'preCrop': False, 'padDistance': 5, 'distances': [1], 'force2D': False, 'force2Ddimension': 0, 'resegmentRange': None, 'label': 255, 'additionalInfo': True, 'binWidth': 25, 'weightingNorm': None}
diagnostics_Configuration_EnabledImageTypes: {'Original': {}}
diagnostics_Image-original_Hash: 6b16bcf3ed3536f520b88973c518a579900713f0
diagnostics_Image-original_Dimensionality: 3D
diagnostics_Image-original_Spacing: (0.68359375, 0.68359375, 1.0)
diagnostics_Image-original_Size: (512, 512, 343)
diagnostics_Image-original_Mean: -583.0575106540157
diagnostics_Image-original_Minimum: -1024.0
di

In [30]:
features.items()


odict_items([('diagnostics_Versions_PyRadiomics', '0+unknown'), ('diagnostics_Versions_Numpy', '1.26.2'), ('diagnostics_Versions_SimpleITK', '2.1.1.2'), ('diagnostics_Versions_PyWavelet', '1.2.0'), ('diagnostics_Versions_Python', '3.10.13'), ('diagnostics_Configuration_Settings', {'minimumROIDimensions': 2, 'minimumROISize': None, 'normalize': False, 'normalizeScale': 1, 'removeOutliers': None, 'resampledPixelSpacing': None, 'interpolator': 'sitkBSpline', 'preCrop': False, 'padDistance': 5, 'distances': [1], 'force2D': False, 'force2Ddimension': 0, 'resegmentRange': None, 'label': 255, 'additionalInfo': True, 'binWidth': 25, 'weightingNorm': None}), ('diagnostics_Configuration_EnabledImageTypes', {'Original': {}}), ('diagnostics_Image-original_Hash', '6b16bcf3ed3536f520b88973c518a579900713f0'), ('diagnostics_Image-original_Dimensionality', '3D'), ('diagnostics_Image-original_Spacing', (0.68359375, 0.68359375, 1.0)), ('diagnostics_Image-original_Size', (512, 512, 343)), ('diagnostics_Im